In [7]:
import asyncio

from h2o_wave import main, app, Q, ui, on, run_on, data, Expando
from typing import Optional, List
import logging

import pandas as pd
import numpy as np

import cards
import utils

conn = utils.TimedSQLiteConnection('UMGC.db')

user_role = 'student'
user_id = 6
timed_connection = conn

In [8]:
a = Expando({'1': None, '2': None, 'nopre': False})

In [12]:
async def initialize_ge():
    """
    Initialize General Education tracking for undergraduate students.
    """
    return {
        'arts': {'1': None, '2': None, 'nopre': False},
        'beh': {'1': None, '2': None, 'nopre': False},
        'bio': {'1a': None, '1b': None, '1c': None, '2': None, 'nopre': False},
        'comm': {'1': 'WRTG 111', '2': 'WRTG 112', '3': None, '4': None, 'nopre': False},
        'math': {'1': None, 'nopre': False},
        'res': {'1': None, '2': 'LIBS 150', '3': None, '3a': None, '3b': None, '3c': None, 'nopre': False}
    }

In [13]:
await initialize_ge()

{'arts': {'1': None, '2': None, 'nopre': False},
 'beh': {'1': None, '2': None, 'nopre': False},
 'bio': {'1a': None, '1b': None, '1c': None, '2': None, 'nopre': False},
 'comm': {'1': 'WRTG 111',
  '2': 'WRTG 112',
  '3': None,
  '4': None,
  'nopre': False},
 'math': {'1': None, 'nopre': False},
 'res': {'1': None,
  '2': 'LIBS 150',
  '3': None,
  '3a': None,
  '3b': None,
  '3c': None,
  'nopre': False}}

In [3]:
from tools.timedsqlite import get_query_one, get_query_dict
from utils import populate_student_info_dict, populate_student_data_dict, get_required_program_courses, get_student_progress_d3

student_info = await populate_student_info_dict(timed_connection, user_id)
student_data = await populate_student_data_dict(timed_connection, student_info)

In [5]:
student_data['required']

,id,course,type,title,credits,pre,pre_credits,substitutions,description
0,22,BMGT 110,Major,Introduction to Business and Management,3,,,,(For students with little or no business backg...
1,23,ACCT 220,Major,Principles of Accounting I,3,,,,An introduction to the basic theory and techni...
2,24,ACCT 221,Major,Principles of Accounting II,3,ACCT 220,,,Prerequisite: ACCT 220. Further study of conte...
3,25,BMGT 364,Major,Management and Organization Theory,3,,,,Recommended: BMGT 110. An examination of the f...
4,26,BMGT 365,Major,Organizational Leadership,3,(BMGT 110 | BMGT 364),,,Prerequisite: BMGT 110 or BMGT 364. An explora...
5,27,MRKT 310,Major,Marketing Principles,3,,,,A foundational study of the marketing principl...
6,28,BMGT 380,Major,Business Law I,3,,,,(Strongly recommended for students seeking car...
7,29,HRMN 300,Major,Human Resource Management,3,,,,A basic study of the strategic role of human r...
8,30,FINC 330,Major,Business Finance,3,(ACCT 221 & STAT 200),,,Prerequisites: ACCT 221 and STAT 200. An overv...
9,31,BMGT 496,Major,Business Ethics,3,,,,A study of the relationship of business ethics...


In [6]:
student_data['schedule']

,id,user_id,seq,course,title,credits,course_type,type,completed,term,session,locked,prerequisites,pre,pre_credits,substitutions,description
0,83,6,1,PACE 111B,Program and Career Exploration in Business,3,general,general,0,1,1,1,,,,"PACE 111B, PACE 111C, PACE 111M, PACE 111P, PA...",(Fulfills the general education requirement in...
1,84,6,2,LIBS 150,Introduction to Research,1,general,general,0,1,1,1,,,,"COMP 111, LIBS 100, or LIBS 150",An introduction to the research process and me...
2,85,6,3,WRTG 111,Academic Writing I,3,general,general,0,1,3,0,,,,"WRTG 100A, WRTG 111, or WRTG 111X",(The first course in the two-course series WRT...
3,86,6,4,NUTR 100,Elements of Nutrition,3,general,general,0,1,3,0,,,,NUTR 100 or NUTR 200,A study of the scientific and quantitative fou...
4,87,6,5,WRTG 112,Academic Writing II,3,general,general,0,2,1,0,,,,"ENGL 101, ENGL 101X, WRTG 101, WRTG 101S, WRTG...",(The second course in the two-course series WR...
5,88,6,6,BMGT 110,Introduction to Business and Management,3,major,major,0,2,2,0,,,,,(For students with little or no business backg...
6,89,6,7,SPCH 100,Foundations of Oral Communication,3,general,general,0,3,1,0,,,,"SPCH 100, SPCH 100X, SPCH 101, SPCH 107, or SP...","For online sections, access to a broadband int..."
7,90,6,8,ELECTIVE,,3,elective,elective,0,3,1,0,,,,,
8,91,6,9,STAT 200,Introduction to Statistics,3,"required,general",required,0,3,3,0,,,,"BEHS 202, BEHS 302, BMGT 230, ECON 321, GNST 2...",An introduction to statistics. The objective i...
9,92,6,10,IFSM 300,Information Systems in Organizations,3,"required,general",required,0,3,3,0,,,,,An overview of information systems and how the...


In [ ]:
program_id = student_info['program_id']
if program_id is not None:
    student_data['required'] = await get_required_program_courses(timed_connection, program_id)

In [ ]:
student_data['required']

In [ ]:
app_stage_id = student_info['app_stage_id']
if app_stage_id == 4:
    student_data['schedule'] = await get_student_progress_d3(timed_connection, user_id)

In [ ]:
attributes = ['resident_status', 'app_stage_id', 'app_stage', 'student_profile', 'financial_aid', 
        'transfer_credits', 'program_id']
query = '''
SELECT user_id, fullname AS name, resident_status, app_stage_id, app_stage, student_profile,
        transfer_credits, financial_aid, program_id
    FROM student_info_view WHERE user_id = ?
'''

row = await conn.fetchone(query, params=(user_id,))
student_info = {name: row[name] for name in attributes}
student_info

In [ ]:
from tools.timedsqlite import get_query_df
from utils import get_student_progress_d3

df = await get_student_progress_d3(timedConnection, user_id)
df.head()

In [ ]:
attributes = ['resident_status', 'app_stage_id', 'app_stage', 'student_profile', 'financial_aid', 'transfer_credits', 'program_id']
query = '''
    SELECT user_id, fullname AS name, resident_status, app_stage_id, app_stage, student_profile, transfer_credits, financial_aid, program_id
    FROM student_info_view WHERE user_id = ?
'''
tmp = await get_query_dict(timedConnection, query, params=(user_id,))
student_info = tmp[0]
student_info

In [ ]:
program_id = student_info['program_id']
program_id

In [ ]:
from utils import get_required_program_courses, get_program_title

In [ ]:
if program_id is not None:
    row = await get_program_title(timedConnection, program_id)
    if row:
        student_info['degree_program'] = row['title']
        student_info['degree_id'] = row['id']
    student_data_required = await get_required_program_courses(timedConnection, program_id)

student_info

In [ ]:
student_data_required

In [ ]:

        if program_id is not None:
            row = await get_program_title(timedConnection, program_id)
            if row:
                q.user.student_info['degree_program'] = row['title']
                q.user.student_info['degree_id'] = row['id']
            q.user.student_data['required'] = await get_required_program_courses(timedConnection, program_id)

        if q.user.student_info['app_stage_id'] == 4:
            q.user.student_data['schedule'] = await get_student_progress_d3(timedConnection, user_id)
    
        if q.user.student_info['first_term'] is None:
            q.user.student_info['first_term'] = q.app.default_first_term

    # Recreate dropdown menus for students
    # Need to do this only if dropdown menu status was not saved
    # (We should save this status in the future)

    program_id = q.user.student_info['program_id']
    if program_id is not None:
        # recreate dropdown menu for program if empty
        if q.user.student_info['menu']['program'] is None:
            q.user.student_info['menu']['program'] = program_id
        # recreate dropdowns for degree and area_of_study if either is empty
        if (q.user.student_info['menu']['degree'] is None) or (q.user.student_info['menu']['area_of_study'] is None):
            query = '''
                SELECT menu_degree_id, menu_area_id 
                FROM menu_all_view
                WHERE program_id = ?
                LIMIT 1
            '''
            # limit 1 because there is not a strict 1:1 correspondence between study areas and programs
            row = await get_query_one(timedConnection, query, params=(program_id,))
            if row:
                q.user.student_info['menu']['degree'] = row['menu_degree_id']
                q.user.student_info['menu']['area_of_study'] = row['menu_area_id']

In [ ]:
async def populate_student_info(timedConnection, user_id):
    '''
    Get information from student_info table and populate the 
    q.user.student_info variables and 
    q.user.student_data dataframes

    Called by `app.initialize_user` and `app.select_sample_user`

    Create a dictionary first named student_info
    
    Note: timedConnection and user_id also stored in q. Included in parameters for clarity.
    '''

    attributes = ['resident_status', 'app_stage_id', 'app_stage', 'student_profile', 'financial_aid', 
        'transfer_credits', 'program_id']
    query = '''
    SELECT user_id, fullname AS name, resident_status, app_stage_id, app_stage, student_profile,
        transfer_credits, financial_aid, program_id
    FROM student_info_view WHERE user_id = ?
    '''
    row = await get_query_one(timedConnection, query, params=(user_id,))
    
    if row:
        q.user.student_info.update({name: row[name] for name in attributes})
        q.user.student_info['user_id'] = user_id
        q.user.student_info['name'] = row['name']
        q.user.student_data['user_id'] = user_id
        program_id = q.user.student_info['program_id']

        if program_id is not None:
            row = await get_program_title(timedConnection, program_id)
            if row:
                q.user.student_info['degree_program'] = row['title']
                q.user.student_info['degree_id'] = row['id']
            q.user.student_data['required'] = await get_required_program_courses(timedConnection, program_id)

        if q.user.student_info['app_stage_id'] == 4:
            q.user.student_data['schedule'] = await get_student_progress_d3(timedConnection, user_id)
    
        if q.user.student_info['first_term'] is None:
            q.user.student_info['first_term'] = q.app.default_first_term

    # Recreate dropdown menus for students
    # Need to do this only if dropdown menu status was not saved
    # (We should save this status in the future)

    program_id = q.user.student_info['program_id']
    if program_id is not None:
        # recreate dropdown menu for program if empty
        if q.user.student_info['menu']['program'] is None:
            q.user.student_info['menu']['program'] = program_id
        # recreate dropdowns for degree and area_of_study if either is empty
        if (q.user.student_info['menu']['degree'] is None) or (q.user.student_info['menu']['area_of_study'] is None):
            query = '''
                SELECT menu_degree_id, menu_area_id 
                FROM menu_all_view
                WHERE program_id = ?
                LIMIT 1
            '''
            # limit 1 because there is not a strict 1:1 correspondence between study areas and programs
            row = await get_query_one(timedConnection, query, params=(program_id,))
            if row:
                q.user.student_info['menu']['degree'] = row['menu_degree_id']
                q.user.student_info['menu']['area_of_study'] = row['menu_area_id']

In [ ]:
async def populate_student_info(q, timedConnection, user_id):
    '''
    Get information from student_info table and populate the q.user.student_info variables
    and q.user.student_data dataframes

    Called by `app.initialize_user` and `app.select_sample_user`

    Note: timedConnection and user_id also stored in q. Included in parameters for clarity.
    '''

    attributes = ['resident_status', 'app_stage_id', 'app_stage', 'student_profile', 'financial_aid', 
        'transfer_credits', 'program_id']
    query = '''
    SELECT user_id, fullname AS name, resident_status, app_stage_id, app_stage, student_profile,
        transfer_credits, financial_aid, program_id
    FROM student_info_view WHERE user_id = ?
    '''
    row = await get_query_one(timedConnection, query, params=(user_id,))
    if row:
        q.user.student_info.update({name: row[name] for name in attributes})
        q.user.student_info['user_id'] = user_id
        q.user.student_info['name'] = row['name']
        q.user.student_data['user_id'] = user_id
        program_id = q.user.student_info['program_id']

        if program_id is not None:
            row = await get_program_title(timedConnection, program_id)
            if row:
                q.user.student_info['degree_program'] = row['title']
                q.user.student_info['degree_id'] = row['id']
            q.user.student_data['required'] = await get_required_program_courses(timedConnection, program_id)

        if q.user.student_info['app_stage_id'] == 4:
            q.user.student_data['schedule'] = await get_student_progress_d3(timedConnection, user_id)
    
        if q.user.student_info['first_term'] is None:
            q.user.student_info['first_term'] = q.app.default_first_term

    # Recreate dropdown menus for students
    # Need to do this only if dropdown menu status was not saved
    # (We should save this status in the future)

    program_id = q.user.student_info['program_id']
    if program_id is not None:
        # recreate dropdown menu for program if empty
        if q.user.student_info['menu']['program'] is None:
            q.user.student_info['menu']['program'] = program_id
        # recreate dropdowns for degree and area_of_study if either is empty
        if (q.user.student_info['menu']['degree'] is None) or (q.user.student_info['menu']['area_of_study'] is None):
            query = '''
                SELECT menu_degree_id, menu_area_id 
                FROM menu_all_view
                WHERE program_id = ?
                LIMIT 1
            '''
            # limit 1 because there is not a strict 1:1 correspondence between study areas and programs
            row = await get_query_one(timedConnection, query, params=(program_id,))
            if row:
                q.user.student_info['menu']['degree'] = row['menu_degree_id']
                q.user.student_info['menu']['area_of_study'] = row['menu_area_id']